In [ ]:
#Import Environments:

import pandas as pd
import numpy as np

In [ ]:
#Load Data:

train = pd.read_excel (r'C:\Users\nvhuy\Desktop\test_data.xlsx')
test  = pd.read_excel (r'C:\Users\nvhuy\Desktop\test_data.xlsx')

In [ ]:
train.info(), test.info()

In [ ]:
train.isnull().sum()

In [ ]:
#test['ARR_DELAY'] = test['ARR_DELAY'].apply(lambda x: x/10)
#test.head()

In [ ]:
#test.ARR_DELAY = test.ARR_DELAY.round()
#test.head()

In [ ]:
#test['ARR_DELAY'] = test['ARR_DELAY'].apply(lambda x: x*10)
#test.head()

In [ ]:
train.set_index("CRS_DEP_TIME", inplace=True)
test.set_index("CRS_DEP_TIME", inplace=True)

In [ ]:
train_y = train["ARR_DELAY"]

In [ ]:
#Delete the column ARR_DEL15 from train set:

train.drop(labels="ARR_DELAY", axis=1, inplace=True)

In [ ]:
train.shape, test.shape

In [ ]:
train_test = train.append(test)

In [ ]:
#Delete columns that are not used as features for training and prediction:

columns_to_drop = ["ORIGIN", "DEST"]
train_test.drop(labels=columns_to_drop, axis=1, inplace=True)

In [ ]:
#Replace NaN with zeroes:

train_test.fillna(value=0, inplace=True)

In [ ]:
train_test.isnull().values.any()

In [ ]:
#Generate feature sets (x):

train_x = train_test.values[0:4997]
test_x = train_test.values[4997:]

In [ ]:
train_x.shape, test_x.shape

In [ ]:
#Transform the data:

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_x_scale = scaler.fit_transform(train_x)
test_x_scale = scaler.transform(test_x)

In [ ]:
#Split training feature and target sets into training and validation subsets:
from sklearn.model_selection import train_test_split

train_x_sub, validation_x_sub, train_y_sub, validation_y_sub = train_test_split(train_x_scale, train_y, random_state=0)

In [ ]:
#Train with Gradient Boosting Algorithm:

from sklearn.ensemble import GradientBoostingClassifier

learning_rates = [0.05, 0.01, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gboost = GradientBoostingClassifier(n_estimators=200, learning_rate = learning_rate, max_features=10, max_depth=20, random_state = 0)
    gboost.fit(train_x_sub, train_y_sub)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gboost.score(train_x_sub, train_y_sub)))
    print("Accuracy score (validation): {0:.3f}".format(gboost.score(validation_x_sub, validation_y_sub)))
    print()

In [ ]:
#Output confusion matrix and classification report of Gradient Boosting algorithm on validation set:

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

gboost = GradientBoostingClassifier(n_estimators=20, learning_rate=0.75, max_features=3, max_depth=3, random_state = 0)
gboost.fit(train_x_sub, train_y_sub)
predictions = gboost.predict(validation_x_sub)

print("Confusion Matrix:")
print(confusion_matrix(validation_y_sub, predictions))
print()
print("Classification Report:")
print(classification_report(validation_y_sub, predictions))

In [ ]:
y_pred = gboost.predict(validation_x_sub)

In [ ]:
df=pd.DataFrame({'Actual':validation_y_sub, 'Predicted':y_pred})

In [ ]:
df

In [ ]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(validation_y_sub, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(validation_y_sub, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(validation_y_sub, y_pred)))

In [ ]:
import itertools
import matplotlib.pyplot as plt

In [ ]:
def plot_confusion_matrix(cm,
                          normalize = False,
                          title = 'Confusion Matrix',
                          cmap = plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.ylabel('ACTUAL')
    plt.xlabel('PREDICTED')
    plt.tight_layout()
    plt.grid('off')
    x_labels = ['on-time','','delay']
    y_labels = ['','on-time','','','','delay']
    ax = fig.add_subplot(111)
    ax.set_xticklabels([' '] + x_labels)
    ax.set_yticklabels([' '] + y_labels)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    

# Compute confusion matrix
cnf_matrix = confusion_matrix(validation_y_sub, predictions)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
fig = plt.figure()
plot_confusion_matrix(cnf_matrix, title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
fig = plt.figure()
plot_confusion_matrix(cnf_matrix, normalize=True,
                      title='Normalized confusion matrix')


plt.show()

In [ ]:
# ROC curve and Area-Under-Curve (AUC)

y_scores_gboost = gboost.decision_function(validation_x_sub)
fpr_gboost, tpr_gboost, _ = roc_curve(validation_y_sub, y_scores_gboost)
roc_auc_gboost = auc(fpr_gboost, tpr_gboost)

print("Area under ROC curve = {:0.2f}".format(roc_auc_gboost))